In [32]:
import pandas as pd

train = pd.read_csv('./input/dataset/new_train.csv')
test = pd.read_csv('./input/dataset/new_test.csv')

In [33]:
import os
from PIL import Image

def get_img_info(dir, file):
    file = str(int(file)).zfill(4) + ".jpg"
    img_path = os.path.join(dir, file)
    
    image = Image.open(img_path)
    width, height = image.width, image.height
    
    return width, height

In [1]:
train['image_width'], train['image_height'] = zip(*train.apply(
    lambda row: get_img_info("./input/dataset/train/", row['filename']), axis=1)
)

test['image_width'], test['image_height'] = zip(*test.apply(
    lambda row: get_img_info("input/dataset/test/", row['filename']), axis=1)
)

NameError: name 'train' is not defined

In [35]:
all = pd.concat([train, test]).reset_index(drop=True)
all[:10]

,index,filename,label,center_x,center_y,width,height,image_width,image_height
0,0,1,1,0.438281,0.529551,0.814063,0.704492,640,423
1,1,2,3,0.541016,0.524324,0.849219,0.672973,1992,1328
2,2,3,2,0.492500,0.517045,0.695000,0.725000,600,440
3,3,4,2,0.585156,0.525000,0.692187,0.785135,1600,1065
4,4,5,3,0.385547,0.645270,0.614844,0.698649,800,450
5,5,5,3,0.559375,0.682432,0.445313,0.629730,800,450
6,6,5,3,0.143359,0.382432,0.286719,0.318919,800,450
7,7,6,1,0.429688,0.596737,0.590625,0.778555,640,429
8,8,7,3,0.516797,0.504054,0.380469,0.929730,960,640
9,9,8,0,0.476526,0.520833,0.298122,0.404167,852,480


In [53]:
all['center_x_actual'] = all['center_x'] * all['image_width']
all['center_y_actual'] = all['center_y'] * all['image_height']
all['box_width'] = all['width'] * all['image_width']
all['box_height'] = all['height'] * all['image_height']
all['box_area'] = (all['box_width'] * all['box_height']).round(2)

all['box_left_top_x'] = (all['center_x'] * all['image_width']) - (all['width'] * all['image_width'])/2
all['box_left_top_y'] = (all['center_y'] * all['image_height']) - (all['height'] * all['image_height'])/2
all.head()

,index,filename,label,center_x,center_y,width,height,image_width,image_height,center_x_actual,center_y_actual,box_width,box_height,box_area,box_left_top_x,box_left_top_y,box_right_bottom_x,box_right_bottom_y
0,0,1,1,0.438281,0.529551,0.814063,0.704492,640,423,280.499840,224.000073,521.000320,298.000116,155258.16,19.999680,75.000015,541.000000,373.000131
1,1,2,3,0.541016,0.524324,0.849219,0.672973,1992,1328,1077.703872,696.302272,1691.644248,893.708144,1511836.24,231.881748,249.448200,1923.525996,1143.156344
2,2,3,2,0.492500,0.517045,0.695000,0.725000,600,440,295.500000,227.499800,417.000000,319.000000,133023.00,87.000000,67.999800,504.000000,386.999800
3,3,4,2,0.585156,0.525000,0.692187,0.785135,1600,1065,936.249600,559.125000,1107.499200,836.168775,926056.25,382.500000,141.040613,1489.999200,977.209388
4,4,5,3,0.385547,0.645270,0.614844,0.698649,800,450,308.437600,290.371500,491.875200,314.392050,154641.65,62.500000,133.175475,554.375200,447.567525


In [54]:
train_coco = all[all["filename"] <= 1204].reset_index().copy().drop(columns=['level_0','index'])
test_coco = all[all["filename"] > 1204].reset_index().copy().drop(columns=['level_0','index'])

In [55]:
train_coco.head()

,filename,label,center_x,center_y,width,height,image_width,image_height,center_x_actual,center_y_actual,box_width,box_height,box_area,box_left_top_x,box_left_top_y,box_right_bottom_x,box_right_bottom_y
0,1,1,0.438281,0.529551,0.814063,0.704492,640,423,280.499840,224.000073,521.000320,298.000116,155258.16,19.999680,75.000015,541.000000,373.000131
1,2,3,0.541016,0.524324,0.849219,0.672973,1992,1328,1077.703872,696.302272,1691.644248,893.708144,1511836.24,231.881748,249.448200,1923.525996,1143.156344
2,3,2,0.492500,0.517045,0.695000,0.725000,600,440,295.500000,227.499800,417.000000,319.000000,133023.00,87.000000,67.999800,504.000000,386.999800
3,4,2,0.585156,0.525000,0.692187,0.785135,1600,1065,936.249600,559.125000,1107.499200,836.168775,926056.25,382.500000,141.040613,1489.999200,977.209388
4,5,3,0.385547,0.645270,0.614844,0.698649,800,450,308.437600,290.371500,491.875200,314.392050,154641.65,62.500000,133.175475,554.375200,447.567525


In [56]:
all[all['filename']==336]

,index,filename,label,center_x,center_y,width,height,image_width,image_height,center_x_actual,center_y_actual,box_width,box_height,box_area,box_left_top_x,box_left_top_y,box_right_bottom_x,box_right_bottom_y
580,580,336,3,0.399297,0.60625,0.793911,0.784375,427,640,170.499819,388.0,338.999997,502.0,170178.0,0.99982,137.0,339.999818,639.0


In [57]:
info_dict = {"info": {
    "year":"2022","version":"1",
    "description":"Exported from roboflow.ai",
    "contributor":"",
    "url":"https://public.roboflow.ai/object-detection/undefined","date_created":"2022-12-17T02:13:57+00:00"
}}

license_dict = {
    "licenses":[{"id":1,"url":"https://creativecommons.org/licenses/by/4.0/","name":"CC BY 4.0"}]
}

categories_dict = {
        "categories":[
        {"id": int(0), "name":"Water buffalo","supercategory":"animal"},
        {"id":int(1), "name":"Elephant","supercategory":"animal"},
        {"id":int(2), "name":"Rhinoceros","supercategory":"animal"},
        {"id":int(3), "name":"Zebra","supercategory":"animal"}
    ]
}

In [58]:
images_dict = {
    "images": []
}

annotations_dict = {
    "annotations": []
}

In [59]:
image_set = set()

for idx in range(len(train_coco)):
    tmp_series = train_coco.iloc[idx]

    if int(tmp_series['filename']) in image_set:
        continue
    else:
        tmp_image_dict = {
        "id": int(tmp_series['filename']),
        "file_name": str(int(tmp_series['filename'])).zfill(4) + ".jpg",
        "height": int(tmp_series['image_height']),
        "width": int(tmp_series['image_width'])
        }
        images_dict["images"].append(tmp_image_dict)
    
    tmp_annotation_dict = {
        "id": idx,
        "image_id": int(tmp_series['filename']),
        "category_id": int(tmp_series['label']),
        "bbox":[
            int(tmp_series['box_left_top_x']),
            int(tmp_series['box_left_top_y']),
            int(tmp_series['box_width']),
            int(tmp_series['box_height'])
        ],
        "area": int(tmp_series['box_area']),
    }
    annotations_dict["annotations"].append(tmp_annotation_dict)

In [60]:
annotation_train_dict = {}
annotation_train_dict.update(info_dict)
annotation_train_dict.update(license_dict)
annotation_train_dict.update(categories_dict)
annotation_train_dict.update(images_dict)
annotation_train_dict.update(annotations_dict)

In [61]:
import json

with open("./input/dataset/_annotations.coco.json", "w") as json_file:
    json.dump(annotation_train_dict, json_file)

In [62]:
images_dict = {
    "images": []
}

annotations_dict = {
    "annotations": []
}

In [63]:
image_set = set()

for idx in range(len(test_coco)):
    tmp_series = test_coco.iloc[idx]

    if int(tmp_series['filename']) in image_set:
        continue
    else:
        tmp_image_dict = {
        "id": int(tmp_series['filename']),
        "file_name": str(int(tmp_series['filename'])).zfill(4) + ".jpg",
        "height": int(tmp_series['image_height']),
        "width": int(tmp_series['image_width'])
        }
        images_dict["images"].append(tmp_image_dict)
    
    tmp_annotation_dict = {
        "id": idx,
        "image_id": int(tmp_series['filename']),
        "category_id": int(tmp_series['label']),
        "bbox":[
            int(tmp_series['box_left_top_x']),
            int(tmp_series['box_left_top_y']),
            int(tmp_series['box_width']),
            int(tmp_series['box_height'])
        ],
        "area": int(tmp_series['box_area']),
    }
    annotations_dict["annotations"].append(tmp_annotation_dict)

In [64]:
annotation_train_dict = {}
annotation_train_dict.update(info_dict)
annotation_train_dict.update(license_dict)
annotation_train_dict.update(categories_dict)
annotation_train_dict.update(images_dict)
annotation_train_dict.update(annotations_dict)

In [65]:
import json

with open("./input/dataset/_annotations.coco.json", "w") as json_file:
    json.dump(annotation_train_dict, json_file)